<a href="https://colab.research.google.com/github/KusalaniR/MedGen.AI/blob/main/notebooks/Merging_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#Import Libraries
import pandas as pd


In [ ]:
#Load All Datasets
import pandas as pd

knowledge_df = pd.read_csv(
    '/content/drive/MyDrive/MedGen.AI Datasets/FINALIZED DATASETS/blood_test_knowledge.csv',
    encoding='ISO-8859-1'  # alternative encoding
)

mapping_df = pd.read_csv(
    '/content/drive/MyDrive/MedGen.AI Datasets/FINALIZED DATASETS/blood_test_mapping.csv',
    encoding='ISO-8859-1'
)

rules_df = pd.read_csv(
    '/content/drive/MyDrive/MedGen.AI Datasets/FINALIZED DATASETS/blood_test_rules.csv',
    encoding='ISO-8859-1'
)

blood_df = pd.read_csv(
    '/content/drive/MyDrive/MedGen.AI Datasets/FINALIZED DATASETS/labevents.csv',
    encoding='ISO-8859-1'
)



In [ ]:
#merge datasets
#Merge mapping to get test_name in labevents
blood_df = blood_df.merge(mapping_df, on='itemid', how='left')


In [ ]:
#Merge knowledge (explanations)
blood_df = blood_df.merge(knowledge_df, on='test_name', how='left')


In [ ]:
#Merge rules (thresholds)
blood_df['gender'] = 'Any'  # temporary, for merging rules


In [ ]:
blood_df = blood_df.merge(rules_df, on=['test_name', 'gender'], how='left')


In [ ]:
#Compute Normal / Abnormal
def classify(row):
    if row['valuenum'] < row['low_threshold']:
        return 'Low'
    elif row['valuenum'] > row['high_threshold']:
        return 'High'
    else:
        return 'Normal'

blood_df['status'] = blood_df.apply(classify, axis=1)


In [ ]:
#Save merged dataset
blood_df.to_csv("/content/drive/MyDrive/MedGen.AI Datasets/FINALIZED DATASETS/blood_all_merged.csv", index=False)


In [ ]:
#load merged data set
import pandas as pd

merged_df = pd.read_csv(
    '/content/drive/MyDrive/MedGen.AI Datasets/FINALIZED DATASETS/blood_all_merged.csv',
    encoding='latin1'
)


In [ ]:
merged_df.head()


,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,...,normal_range,unit_x,low_meaning,high_meaning,simple_explanation_en,gender,low_threshold,high_threshold,unit_y,status
0,172061,10014354,29600294.0,1808066,51277,NaN,2148-08-16 00:00:00,2148-08-16 01:30:00,15.4,15.40,...,11Â15,%,Usually not significant,Variation in red blood cell size,RDW shows how much your red blood cells differ...,Any,11.0,15.0,%,High
1,172062,10014354,29600294.0,1808066,51279,NaN,2148-08-16 00:00:00,2148-08-16 01:30:00,3.35,3.35,...,"4.2Â5.4 (Female), 4.7Â6.1 (Male)",m/uL,May reduce oxygen delivery,May thicken blood,Red blood cells carry oxygen from your lungs t...,Any,NaN,NaN,NaN,Normal
2,172068,10014354,29600294.0,1808066,52172,NaN,2148-08-16 00:00:00,2148-08-16 01:30:00,49.7,49.70,...,NaN,NaN,NaN,NaN,NaN,Any,NaN,NaN,NaN,Normal
3,172063,10014354,29600294.0,1808066,51301,NaN,2148-08-16 00:00:00,2148-08-16 01:30:00,20.3,20.30,...,NaN,NaN,NaN,NaN,NaN,Any,NaN,NaN,NaN,Normal
4,172050,10014354,29600294.0,1808066,51249,NaN,2148-08-16 00:00:00,2148-08-16 01:30:00,31.1,31.10,...,32Â36,g/dL,Red blood cells may carry less oxygen,Rarely significant,MCHC shows how concentrated hemoglobin is in r...,Any,32.0,36.0,g/dL,Low


In [ ]:
#FILTER ONLY 12 BLOOD TESTS
selected_tests = [
    'Hemoglobin', 'Hematocrit', 'Red Blood Cells', 'WBC Count',
    'Platelet Count', 'MCV', 'MCH', 'MCHC', 'RDW',
    'Cholesterol, Total', 'Triglycerides', 'Glucose'
]

filtered_df = merged_df[merged_df['test_name'].isin(selected_tests)]


In [ ]:
#check result
filtered_df[['test_name', 'valuenum', 'status']].head(10)


,test_name,valuenum,status
0,RDW,15.40,High
1,Red Blood Cells,3.35,Normal
4,MCHC,31.10,Low
5,Hematocrit,29.60,Normal
6,Hemoglobin,9.20,Normal
7,Platelet Count,206.00,Normal
8,MCV,88.00,Normal
9,MCH,27.50,Normal
30,Glucose,193.00,High
80,Glucose,210.00,High


In [ ]:
#save final blood test dataset only with 12 tests
filtered_df.to_csv(
    '/content/drive/MyDrive/MedGen.AI Datasets/FINALIZED DATASETS/final_merged_all_blood_tests_dataset.csv',
    index=False
)
